In [1]:
# Import dependencies
import pandas as pd
import numpy as np

In [2]:
# Read original database
hw_df = pd.read_csv('US_counties_COVID19_health_weather_data.csv')

C:\Users\spo.DT\Anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (190,202) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# Find the most recent date of data collection
max_date = hw_df['date'].max()

In [4]:
# Make sure all the counties have info for most recent date of data collection
max_date == hw_df[['fips','date']].groupby('fips').max().reset_index()['date'].unique()[0]

True

In [5]:
# Filter the original dataframe to the max_date
hw_date = hw_df[hw_df['date'] == max_date].reset_index(drop=True)
len(hw_date)

2942

In [6]:
# Find US average for each parameter - if applicable
col_w_US_mean = pd.DataFrame(hw_date.mean()).dropna().index.tolist()
col_wo_US_mean = set(hw_date.columns) - set(col_w_US_mean)
hw_US_mean_df = pd.DataFrame(hw_date[col_w_US_mean].mean()).T

In [11]:
# Group data by state, and get the average value
hw_state = hw_date.groupby(['state']).mean().reset_index()

In [20]:
# Fill the null values in state averages with national averages
columns = hw_US_mean_df.columns.to_list()
columns.remove('presence_of_water_violation')
for col in columns:
    hw_state[col].fillna(hw_US_mean_df[col][0], inplace=True)

In [98]:
# Fill the null values in date filtered dataframe with state averages dataframe
hw_merged = hw_date.merge(hw_state,on='state', how='left')
x_col = [col for col in hw_merged.columns if col.endswith('_x')==True]
y_col = [col for col in hw_merged.columns if col.endswith('_y')==True]

for col in x_col:
    hw_merged[col] = np.where(hw_merged[col].isnull()==False, hw_merged[col], hw_merged[col.replace("_x","_y")])
    
hw_merged.drop(columns=y_col, inplace=True)
hw_merged.columns = [col.replace("_x","") for col in hw_merged.columns.to_list()]

In [99]:
# QC on the process to make sure all the null values has been replaced except for the columns with all values as NA values
hw_merged_na = []
for col in hw_merged.columns:
    if hw_merged[col].isnull().nunique() == 1 and hw_merged[col].isnull().unique()[0] == False:
        continue
    else:
        hw_merged_na.append(col)
        
set(hw_merged_na)-set(col_wo_US_mean)
# 'presence_of_water_violation' is a boolean field - so out of the game!
hw_merged_na.remove('presence_of_water_violation')

In [100]:
hw_merged_na

['CALL',
 'station_id',
 'station_name',
 'mean_temp',
 'min_temp',
 'max_temp',
 'dewpoint',
 'sea_level_pressure',
 'station_pressure',
 'visibility',
 'wind_speed',
 'max_wind_speed',
 'wind_gust',
 'precipitation',
 'precip_flag',
 'fog',
 'rain',
 'snow',
 'hail',
 'thunder',
 'tornado',
 'mean_temp_3d_avg',
 'mean_temp_5d_avg',
 'mean_temp_10d_avg',
 'mean_temp_15d_avg',
 'max_temp_3d_avg',
 'max_temp_5d_avg',
 'max_temp_10d_avg',
 'max_temp_15d_avg',
 'min_temp_3d_avg',
 'min_temp_5d_avg',
 'min_temp_10d_avg',
 'min_temp_15d_avg',
 'dewpoint_3d_avg',
 'dewpoint_5d_avg',
 'dewpoint_10d_avg',
 'dewpoint_15d_avg',
 'date_stay_at_home_announced',
 'date_stay_at_home_effective']

In [73]:
set(col_wo_US_mean)-set(hw_merged_na)

{'county',
 'date',
 'fips',
 'state',
 'stay_at_home_announced',
 'stay_at_home_effective'}

In [89]:
# Drop the columns that are all NA values
#hw_merged.drop(columns=hw_merged_na, inplace=True)

In [103]:
# Replace the columns that are all NA values with "0"
hw_merged.fillna('0', inplace=True)

In [104]:
hw_merged.to_csv('US_counties_COVID19_health_weather_data_replaced_na.csv', index=False)